In [1]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import cv2
import pandas as pd

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from keras_frcnn.mylayer import mylayer
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

K.set_learning_phase(1) #set learning phase

# sys.setrecursionlimit(40000)

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
network ='vgg'

if network == 'vgg':
	from keras_frcnn import vgg as nn
elif network == 'resnet50':
	from keras_frcnn import resnet as nn
else:
	print('Not a valid model')
	raise ValueError


In [3]:
from keras_frcnn.RoiPoolingConv import RoiPoolingConv
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout

if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
else:
	input_shape_img = (224,224, 3)

img_input = Input(shape=input_shape_img)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)


out = (Flatten(name='flatten'))(shared_layers)
out = (Dense(4096, activation='relu', name='fc5'))(out)
out = (Dropout(0.5))(out)
out = (Dense(2048, activation='relu', name='fc6'))(out)
out = (Dropout(0.5))(out)
out = (Dense(512, activation='relu', name='fc7'))(out)
out_class = (Dense(46, activation='sigmoid'))(out)

model_classifier = Model(img_input, out_class)


try:
	print('loading weights from pretrained model')
	model_classifier.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels.h5', by_name=True)
except:
	print('Could not load pretrained model weights. Weights can be found in the keras application folder \
		https://github.com/fchollet/keras/tree/master/keras/applications')

optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_classifier.compile(optimizer=optimizer_classifier, loss = 'binary_crossentropy', metrics= ['accuracy'])



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
loading weights from pretrained model


In [4]:
def name(x):
    name = str.split(x,'/')
    return '/Users/1024329/Downloads/DeepFashion/img_n/' + name[1] + '/' + name[2]
    
df3 = pd.read_pickle('upper_body_atts.pkl')
df3['img_name'] = df3['img_name'].apply(lambda x:name(x))

In [5]:
df3 = df3[['img_name','landmarks','atts']]
df3.head()

,img_name,landmarks,atts
0,/Users/1024329/Downloads/DeepFashion/img_n/Lon...,"[[96, 2, 60, 60], [165, 2, 60, 60], [300, 153,...","[99, 495, 571, 881, 940]"
1,/Users/1024329/Downloads/DeepFashion/img_n/Hoo...,"[[105, 5, 60, 60], [255, 3, 60, 60], [26, 195,...","[470, 953]"
2,/Users/1024329/Downloads/DeepFashion/img_n/Rea...,"[[126, 36, 60, 60], [244, 42, 60, 60], [67, 3,...","[745, 883]"
3,/Users/1024329/Downloads/DeepFashion/img_n/Flo...,"[[67, 8, 60, 60], [217, 2, 60, 60], [24, 167, ...","[142, 306, 365]"
4,/Users/1024329/Downloads/DeepFashion/img_n/Bea...,"[[51, 68, 60, 60], [280, 80, 60, 60], [25, 13,...","[50, 121, 122, 226, 681]"


In [6]:
# removing the samples without any attribute present in it
sample = df3['atts'].apply(lambda x: x.shape[0]>0)
df4 =df3[sample]
df4.shape, df3.shape
# checking the maximum value of the attribute label
df4['atts'].apply(lambda x: np.max(x)).max()

# reset index of df4
df4 = df4.reset_index(drop = True)

In [7]:
df4.shape

(132290, 3)

In [8]:
vals = np.array([])
for i in range(df4.shape[0]):
    if i%10000 ==0: print i
    vals = np.concatenate([vals,df4.loc[i,'atts']]) 

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


In [9]:
import matplotlib.pyplot as plt
his = plt.hist(vals,bins =1000)
select =  np.nonzero(his[0] > 2000)

In [10]:
select[0].shape

(46,)

In [11]:
def selecting(x,select):
    s = [i for i in x if i in (select[0])]
    return s

In [12]:
df4['select46_atts'] = df4['atts'].apply(lambda x: selecting(x,select))

In [13]:
df4.head()

,img_name,landmarks,atts,select46_atts
0,/Users/1024329/Downloads/DeepFashion/img_n/Lon...,"[[96, 2, 60, 60], [165, 2, 60, 60], [300, 153,...","[99, 495, 571, 881, 940]","[495, 571]"
1,/Users/1024329/Downloads/DeepFashion/img_n/Hoo...,"[[105, 5, 60, 60], [255, 3, 60, 60], [26, 195,...","[470, 953]",[470]
2,/Users/1024329/Downloads/DeepFashion/img_n/Rea...,"[[126, 36, 60, 60], [244, 42, 60, 60], [67, 3,...","[745, 883]","[745, 883]"
3,/Users/1024329/Downloads/DeepFashion/img_n/Flo...,"[[67, 8, 60, 60], [217, 2, 60, 60], [24, 167, ...","[142, 306, 365]","[142, 365]"
4,/Users/1024329/Downloads/DeepFashion/img_n/Bea...,"[[51, 68, 60, 60], [280, 80, 60, 60], [25, 13,...","[50, 121, 122, 226, 681]",[226]


In [14]:
# removing the samples without any attribute present in it
sample = df4['select46_atts'].apply(lambda x: np.array(x).shape[0]>0)
df5 =df4[sample]
print df5.shape, df4.shape
# checking the maximum value of the attribute label
print df5['select46_atts'].apply(lambda x: np.max(x)).max()

# reset index of df4
df5 = df5.reset_index(drop = True)

(106702, 4) (132290, 4)
993


In [15]:
df5['select46_atts'] = df5['select46_atts'].apply(lambda x: np.array(x))

In [16]:
select[0]

array([  1,  93, 112, 142, 162, 181, 196, 207, 212, 226, 227, 254, 310,
       335, 365, 380, 441, 470, 495, 513, 546, 568, 571, 574, 577, 601,
       640, 695, 705, 720, 730, 745, 751, 754, 760, 818, 822, 836, 837,
       883, 884, 892, 935, 956, 983, 993])

In [17]:
df5.shape

(106702, 4)

In [18]:
def get_batch(num):
    img = []
    attribs = []
    b_size = 32
    for i in range(b_size*num, b_size*(num+1)):
        #print i
        a = cv2.imread(df5.loc[i,'img_name'])
        a = cv2.resize(a,(224,224))
        a = np.array(a).reshape(224,224,3)
        img.append(a/255)
        atts = df5.loc[i,'select46_atts']
        labels = np.array([0 if item not in atts else 1 for item in select[0]]).reshape(46,)
        attribs.append(labels)
    return np.array(img), np.array(attribs)

In [19]:
select[0]

array([  1,  93, 112, 142, 162, 181, 196, 207, 212, 226, 227, 254, 310,
       335, 365, 380, 441, 470, 495, 513, 546, 568, 571, 574, 577, 601,
       640, 695, 705, 720, 730, 745, 751, 754, 760, 818, 822, 836, 837,
       883, 884, 892, 935, 956, 983, 993])

In [20]:
df5 = df5.head(35000)

In [21]:
df5 = df5.sample(frac = 1).reset_index(drop =True)

In [ ]:
train_loss = []
# train_acc = []
for j in range(50):
    df5 = df5.sample(frac = 1).reset_index(drop =True)
    print 'epoch number:', j
    los = 0
    acc = 0
    for i in range(1000):
        print 'epoch and batch:', j, i 
        X,Y = get_batch(i)
        history = model_classifier.fit(x=X, y= Y, verbose = 2)
        los = los + np.array(history.history['loss'])
        #acc = acc + np.array(history.history['acc'])
        #print los
    train_loss.append(los)
    model_classifier.save_weights('atts_vgg_model.h5')
    #train_acc.append(acc)
    #print train_loss, train_acc
        

epoch number: 0
epoch and batch: 0 0
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
 - 33s - loss: 0.8568 - acc: 0.4524
epoch and batch: 0 1
Epoch 1/1
 - 25s - loss: 0.7956 - acc: 0.4864
epoch and batch: 0 2
Epoch 1/1
 - 27s - loss: 0.7359 - acc: 0.5442
epoch and batch: 0 3
Epoch 1/1
 - 28s - loss: 0.7055 - acc: 0.5442
epoch and batch: 0 4
Epoch 1/1
 - 30s - loss: 0.6637 - acc: 0.6121
epoch and batch: 0 5
Epoch 1/1
 - 26s - loss: 0.6349 - acc: 0.6325
epoch and batch: 0 6
Epoch 1/1
 - 29s - loss: 0.5964 - acc: 0.6678
epoch and batch: 0 7
Epoch 1/1
 - 27s - loss: 0.5764 - acc: 0.7018
epoch and batch: 0 8
Epoch 1/1
 - 28s - loss: 0.5404 - acc: 0.7357
epoch and batch: 0 9
Epoch 1/1
 - 27s - loss: 0.4996 - acc: 0.7772
epoch and batch: 0 10
Epoch 1/1
 - 28s - loss: 0.4692 - acc: 0.8064
epoch and batch: 0 11
Epoch 1/1
 - 27s - loss: 0.4538 - acc: 0.8207
epoch and batch: 0 12
Epoch 1/1
 - 28s - loss: 0.4141 - acc: 0.8363
epoch and batch: 0 13
Epoch 1/1
 - 26s - loss: 0.3956 - acc: 0

In [45]:
xa,ya = get_batch(1)
out_att = model_classifier.predict(xa)

In [32]:
print np.nonzero(ya[0,:]>0), np.nonzero(out_att[0,:] > 0.5)

(array([23, 46]),) (array([ 5, 46, 51]),)


In [296]:
df5.head()

,img_name,landmarks,atts,select46_atts
0,/Users/1024329/Downloads/DeepFashion/img_n/Lon...,"[[96, 2, 60, 60], [165, 2, 60, 60], [300, 153,...","[99, 495, 571, 881, 940]","[495, 571]"
1,/Users/1024329/Downloads/DeepFashion/img_n/Hoo...,"[[105, 5, 60, 60], [255, 3, 60, 60], [26, 195,...","[470, 953]",[470]
2,/Users/1024329/Downloads/DeepFashion/img_n/Rea...,"[[126, 36, 60, 60], [244, 42, 60, 60], [67, 3,...","[745, 883]","[745, 883]"
3,/Users/1024329/Downloads/DeepFashion/img_n/Flo...,"[[67, 8, 60, 60], [217, 2, 60, 60], [24, 167, ...","[142, 306, 365]","[142, 365]"
4,/Users/1024329/Downloads/DeepFashion/img_n/Bea...,"[[51, 68, 60, 60], [280, 80, 60, 60], [25, 13,...","[50, 121, 122, 226, 681]",[226]


In [38]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
# model_classifier.fit(x = [X,X2],y= Y, batch_size=1, epochs=2,validation_split=0.2,callbacks = [checkpoint])